reference: 

https://share.gishub.org/pakistan_floods/

earthengine.google.com

# Dependencies

In [1]:
#pip install voila

   ---------------------------------------- 0.0/4.5 MB ? eta -:--:--
   ------------- -------------------------- 1.6/4.5 MB 9.3 MB/s eta 0:00:01
   ------------------------------ --------- 3.4/4.5 MB 8.8 MB/s eta 0:00:01
   ---------------------------------------  4.5/4.5 MB 8.7 MB/s eta 0:00:01
   ---------------------------------------- 4.5/4.5 MB 7.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
import pandas as pd

csv_flood_events = pd.read_csv(r'C:\Users\dottorandi\Desktop\KHM\sssss\git notebooks\FindingSakura\FS_flood_events.csv')
csv_flood_events

,flood_event_code,country_name,coverage,center_1,center_2,pre_flood_start_date,pre_flood_end_date,flood_start_date,flood_end_date,source
0,FE1,Pakistan,NaN,29.305500,68.906200,2021-8-1,2021-9-30,2022-08-01,2022-09-30,NaN
1,FE2,Philippines,Naga,13.643539,123.243664,2023-10-22,2023-10-25,2024-10-21,2024-10-24,https://global-flood.emergency.copernicus.eu/n...
2,FE3,Spain,Valencia,39.476393,-0.374734,2023-10-29,2023-11-5,2024-10-28,2024-11-04,https://climate.copernicus.eu/esotc/2024/flooding


# OPEN WATER - Event Viz

### Setup

In [2]:
# Import Libraries

import ee
import geemap.foliumap as geemap

In [3]:
import pandas as pd

csv_event = pd.read_csv(r'C:\Users\dottorandi\Desktop\KHM\sssss\git notebooks\FindingSakura\FindingSakura - FS_events.csv')
csv_event

,flood_event_code,country_name,coverage,center_1,center_2,pre_flood_start_date,pre_flood_end_date,flood_start_date,flood_end_date,event_type,source
0,FE1,Pakistan,NaN,29.305500,68.906200,2021-8-1,2021-9-30,2022-08-01,2022-09-30,flood,NaN
1,FE2,Philippines,Naga,13.643539,123.243664,2023-10-22,2023-10-25,2024-10-21,2024-10-24,flood,https://global-flood.emergency.copernicus.eu/n...
2,FE3,Spain,Valencia,39.476393,-0.374734,2023-10-29,2023-11-5,2024-10-28,2024-11-04,flood,https://climate.copernicus.eu/esotc/2024/flooding


In [4]:
w_event = 'FE1'

# Filter the Flood Event
w_event_row = csv_event.loc[csv_event['flood_event_code'] == w_event]
w_event_row

,flood_event_code,country_name,coverage,center_1,center_2,pre_flood_start_date,pre_flood_end_date,flood_start_date,flood_end_date,event_type,source
0,FE1,Pakistan,NaN,29.3055,68.9062,2021-8-1,2021-9-30,2022-08-01,2022-09-30,flood,NaN


In [5]:
country_name = w_event_row.iloc[0,1]
center = [w_event_row.iloc[0,3], w_event_row.iloc[0,4]]
pre_w_event_start_date = w_event_row.iloc[0,5]
pre_w_event_end_date = w_event_row.iloc[0,6]
w_event_start_date = w_event_row.iloc[0,7]
w_event_end_date = w_event_row.iloc[0,8]

print(f"country: {country_name}\ncenter: {center}\nno event: {pre_w_event_start_date} to {pre_w_event_end_date}\nw/ event: {w_event_start_date} to {w_event_end_date}")

country: Pakistan
center: [29.3055, 68.9062]
no event: 2021-8-1 to 2021-9-30
w/ event: 2022-08-01 to 2022-09-30


In [6]:
# Definitions

dataset_boundary = "USDOS/LSIB_SIMPLE/2017"

# LANDSAT

landsat_preflood_text = "Landsat 2021"
landsat_flood_text = "Landsat 2022"

ndwi_preflood_text = "NDWI 2021"
ndwi_flood_text = "NDWI 2022"

water_preflood_text = "Water 2021"
water_flood_text = "Water 2022"

# SENTINEL 2

sentinel2_preflood_text = "Sentinel-2 2021"
sentinel2_flood_text = "Sentinel-2 2022"


# SAR

sentinel1_preflood_text = "Sentinel-1 2021"
sentinel1_flood_text = "Sentinel-1 2022"

## Sentinel 1

In [7]:
# Create an interactive map

m = geemap.Map(center=center, zoom=6)
m

In [8]:
# Create Sentinel-1 SAR Composites

#Let's filter the COPERNICUS/S1_GRD dataset by the date range and location.

style = {"color": "black", "fillColor": "00000000"}

country = ee.FeatureCollection(dataset_boundary).filter(
    ee.Filter.eq("country_na", country_name)
)

# s1_col_2021
s1_col_preflood = (
    ee.ImageCollection("COPERNICUS/S1_GRD")
    .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VV"))
    .filter(ee.Filter.eq("instrumentMode", "IW"))
    .filter(ee.Filter.eq("orbitProperties_pass", "ASCENDING"))
    .filterDate(pre_w_event_start_date, pre_w_event_end_date)
    .filterBounds(country)
    .select("VV")
)

# s1_col_2022
s1_col_flood = (
    ee.ImageCollection("COPERNICUS/S1_GRD")
    .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VV"))
    .filter(ee.Filter.eq("instrumentMode", "IW"))
    .filter(ee.Filter.eq("orbitProperties_pass", "ASCENDING"))
    .filterDate(w_event_start_date, w_event_end_date)
    .filterBounds(country)
    .select("VV")
)

In [9]:
# Create Sentinel-1 SAR composites for the pre-flood and flood periods.

m = geemap.Map()
m = geemap.Map(center=center, zoom=7)
m.add_basemap("HYBRID")
sar_preflood = s1_col_preflood.reduce(ee.Reducer.percentile([20])).clipToCollection(country)
sar_flood = s1_col_flood.reduce(ee.Reducer.percentile([20])).clipToCollection(country)
m.add_layer(sar_preflood, {"min": -25, "max": -5}, sentinel1_preflood_text)
m.add_layer(sar_flood, {"min": -25, "max": -5}, sentinel1_preflood_text)
m

In [10]:
# Apply speckle filtering

col_preflood = s1_col_preflood.map(lambda img: img.focal_median(100, "circle", "meters"))
col_flood = s1_col_flood.map(lambda img: img.focal_median(100, "circle", "meters"))

m = geemap.Map()
m = geemap.Map(center=center, zoom=7)
m.add_basemap("HYBRID")
sentinel1_preflood = col_preflood.reduce(ee.Reducer.percentile([20])).clipToCollection(country)
sentinel1_flood = col_flood.reduce(ee.Reducer.percentile([20])).clipToCollection(country)
m.add_layer(sentinel1_preflood, {"min": -25, "max": -5}, sentinel1_preflood_text)
m.add_layer(sentinel1_flood, {"min": -25, "max": -5}, sentinel1_flood_text)
m

In [11]:
# Extract SAR water extent

threshold = -18   #why
water_preflood = sar_preflood.lt(threshold) # 1 band image
water_flood = sar_flood.lt(threshold) # 1 band image

m = geemap.Map()
m = geemap.Map(center=center, zoom=8)

m.add_layer(water_preflood.selfMask(), {"palette": "blue"}, water_preflood_text)
m.add_layer(water_flood.selfMask(), {"palette": "red"}, water_flood_text)

m.add_layer(country.style(**style), {}, country_name)
m

In [12]:
# Extract SAR flood extent

flood_extent = water_flood.subtract(water_preflood).gt(0).selfMask()

m = geemap.Map()
m = geemap.Map(center=center, zoom=8)

m.add_layer(sentinel1_preflood, {"min": -25, "max": -5}, sentinel1_preflood_text)
m.add_layer(sentinel1_flood, {"min": -25, "max": -5}, sentinel1_flood_text)

m.add_layer(water_preflood.selfMask(), {"palette": "blue"}, water_preflood_text)
m.add_layer(water_flood.selfMask(), {"palette": "red"}, water_flood_text)

m.add_layer(flood_extent, {"palette": "cyan"}, "Flood Extent")
m.add_layer(country.style(**style), {}, country_name)
m